1. Document classification with LSTM + CNN network (Binary):

In [ ]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from tensorflow.keras.datasets.imdb import load_data, get_word_index
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, Embedding, Conv1D, MaxPool1D, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
warnings.filterwarnings('ignore')
%matplotlib inline

1.1 Read in the data:

In [ ]:
n_words = 3000
(X_train, y_train), (X_test, y_test) = load_data(num_words=n_words)
n_train_size = X_train.shape[0]

In [ ]:
print("-"*50)
print("Training data X shape: {}".format(X_train.shape))
print("Training data y shape: {}".format(y_train.shape))
print("-"*50)
print("Testing data X shape: {}".format(X_test.shape))
print("Testing data y shape: {}".format(y_test.shape))
print("-"*50)

1.2 Explore the data

In [ ]:
# Number of unique values of y = Number of categories of the newswires
n_cat = pd.Series(y_train).nunique()
n_cat

In [ ]:
# Print out an observation (document) contained in X
# It is encoded as integers (indices)
print(X_train[0])

In [ ]:
# Let's check for length of the first 100 documents
# We notice that the length is not uniform
print([len(a) for a in X_train[0:100]])

In [ ]:
# Download the dictionary to translate the indices
my_dict = get_word_index(path='imdb_word_index.json')

In [ ]:
# To view the dictionary
# my_dict

In [ ]:
# Exchange the 'key' and 'value'
my_dict_inv = {v:k for k, v in my_dict.items()}

In [ ]:
# Translate each document
i_review = 10
review = list(pd.Series(X_train[i_review]).apply(lambda x: my_dict_inv[x]))
print(' '.join(review))

1.3 Data preprocessing:

In [ ]:
# Padding: newswire lengths are uniformly matched to maxlen
# Cut awat if longer than maxlen and fill with 0s if shorter than maxlen
X_train = sequence.pad_sequences(X_train, maxlen=100)
X_test = sequence.pad_sequences(X_test, maxlen=100)

1.4 Define the model

In [ ]:
drop_prob = 0.5
n_filters = 64
stride_size = 1
n_neurons = 50
n_input = 100
k_size = 4

In [ ]:
# LSTM + CNN model..
my_model = Sequential()
my_model.add(Embedding(n_words,n_input))           # n_words = vocabulary size, n_input = dimension of the embedding space.
my_model.add(Dropout(rate=drop_prob))
my_model.add(Conv1D(filters=n_filters, kernel_size = k_size, strides=stride_size,padding='valid',activation='relu'))
my_model.add(MaxPool1D(pool_size = 2))
my_model.add(LSTM(units=n_neurons, return_sequences=False, input_shape=(None, n_input), activation='tanh'))
my_model.add(Dense(1, activation='sigmoid'))

In [ ]:
# View the summary.
my_model.summary()

In [ ]:
n_epochs = 5                      # Number of epochs.
batch_size = 50                    # Size of each batch.
learn_rate = 0.002                 # learning rate.

In [ ]:
# Optimizer and compilation.
my_optimizer=Adam(learning_rate=learn_rate)
my_model.compile(loss = "binary_crossentropy", optimizer = my_optimizer, metrics=["accuracy"])

In [ ]:
my_summary = my_model.fit(X_train, y_train, epochs=n_epochs, batch_size = batch_size, validation_split=0.2, verbose = 1)

In [ ]:
plt.plot(my_summary.history['accuracy'], c="b")
plt.plot(my_summary.history['val_accuracy'], c="g")
plt.title('Training History')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower right')
plt.show()

In [ ]:
ACC = my_model.evaluate(X_test, y_test, verbose=0)[1]
print("Test Accuracy : {}".format(np.round(ACC,3)))